<a href="https://colab.research.google.com/github/krits29/regeneron-house-waste/blob/main/regeneron_house_waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('regeneron-house-waste')
jovian.set_colab_id('17GG0OShLzQycPgbwwRcEOjGCWc-vV6Lr')

     |████████████████████████████████| 68 kB 4.0 MB/s 


# regeneron-house-waste

Use the "Run" button to execute the code.

In [2]:
!pip install jovian --upgrade --quiet

In [3]:
import jovian

In [4]:
# Execute this to save new versions of the notebook
jovian.commit(project="regeneron-house-waste")

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/krits29/regeneron-house-waste


'https://jovian.ai/krits29/regeneron-house-waste'

In [5]:
# Using 'opendatasets' and download https://www.kaggle.com/mostafaabla/garbage-classification

In [6]:
!pip install opendatasets --upgrade --quiet

In [7]:
import opendatasets as od

In [8]:
dataset_url = "https://www.kaggle.com/mostafaabla/garbage-classification"

In [9]:
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: krits29
Your Kaggle Key: ··········


  2%|▏         | 5.00M/239M [00:00<00:05, 44.9MB/s]

100%|██████████| 239M/239M [00:02<00:00, 92.4MB/s]


In [10]:
# import dataset into pytorch
# use ImageFolder class from torchvision

In [11]:
data_dir = "./garbage-classification/garbage_classification"

In [12]:
import os

In [13]:
os.listdir(data_dir)

['paper',
 'shoes',
 'battery',
 'trash',
 'clothes',
 'metal',
 'white-glass',
 'brown-glass',
 'green-glass',
 'plastic',
 'biological',
 'cardboard']